In [1038]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import PunktSentenceTokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import regexp_tokenize
from nltk.corpus import wordnet as wn
from nltk.corpus import opinion_lexicon
from nltk.corpus import sentiwordnet as swn
from pywsd import adapted_lesk
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

english_sw = set(stopwords.words("english"))

- **Ejercicio 1:** Se quiere desarrollar un evaluador de sentimientos basado en el uso de un recursos lingüístico externo, en concreto, las listas de palabras de opinión que tiene NLTK.

In [1039]:
opinion1 = "Visceral, stunning and relentless film making. Dicaprio's Herculean, almost purely physical performance" \
            "and Hardy's wide eyed intensity coupled with the almost overwhelming beauty of the landscape - those " \
            "trees, the natural light, the sun peeking through the clouds, rendered the proceedings down to savage" \
            "poetry. A hypnotic, beautiful, exhausting film."

opinion2 = "I saw this film on Friday. For the first 40 minutes involving spoken dialogue they need not have " \
            "bothered. For me the dialogue was totally unintelligible with grunting, southern states drawl, " \
            "and coarse accent that made it impossible to understand what they were saying."

opinion3 = "It was a idiotic film that produces a magnificent fascination."

In [1040]:
def tokenize_text(text):
    sent_tokenized = PunktSentenceTokenizer().tokenize(text)
    word_tokenized = [word_tokenize(s) for s in sent_tokenized]
    return word_tokenized

In [1041]:
def classify(text):
    word_tokenized = tokenize_text(text)
    score = 0
    for s in word_tokenized:
        for w in s:
            if w in opinion_lexicon.positive() and w in opinion_lexicon.negative():
                score += 0
            elif w in opinion_lexicon.positive():
                score += 1
            elif w in opinion_lexicon.negative():
                score -= 1
    if score == 0:
        return "neutro", score
    elif score > 0:
        return "pos", score
    else:
        return "neg", score

In [1042]:
# Opinión 1
print(classify(opinion1))

('pos', 1)


In [1043]:
# Opinión 2
print(classify(opinion2))

('neg', -4)


In [1044]:
# Opinión 3
print(classify(opinion3))

('pos', 1)


---
- **Ejercicio 2:** Se pide lo mismo que en el ejercicio anterior, pero en este caso utilizando SentiWordNet (disponible en NLTK). Esta base de datos proporciona valores positivos y negativos para ciertas palabras en un rango entre -1 y 1. Se puede seguir la misma idea de algoritmo que en el caso anterior, pero hay que tener en cuenta que SentiWordNet nos proporciona puntuaciones para los diferentes sentidos que tiene una palabra. Se puede entonces considerar la puntuación de todos los sentidos de la misma palabra, restando a lo positivo la puntuación negativa. Puede ser interesante que la puntuación global se promedie de acuerdo con el número de sentidos.  Utilizar como entrada las mismas opiniones del ejercicio anterior. ¿El resultado es mejor o peor que el conseguido con el algoritmo del ejercicio 1?

In [1045]:
def classify_swn(text):
    word_tokenized = tokenize_text(text)
    total_score = 0
    for s in word_tokenized:
        for w in s:
            synsets = list(swn.senti_synsets(w))
            word_score = 0
            for synset in synsets:
                word_score += (synset.pos_score() - synset.neg_score())
            if len(synsets) > 0:
                total_score += word_score / len(synsets)
    return total_score

In [1046]:
# Opinión 1
print(classify_swn(opinion1))

0.4219623904464331


In [1047]:
# Opinión 2
print(classify_swn(opinion2))

-0.25598923992673994


In [1048]:
# Opinión 3
print(classify_swn(opinion3))

0.7648809523809523


- **Ejercicio 2.1:** Hacer una variante del ejercicio donde se tengan en cuenta primero la categoría gramatical del token para considerar únicamente los scores de los sentidos que coincidan con la categoría gramatical dada. ¿Ha mejorado el resultado o ha empeorado con respecto a versiones anteriores?

In [1049]:
# Función auxiliar para convertir un
# tag generado por nltk.pos_tag()
# en una categ. gramatical de 
# WordNet
def tag2wn(tag):
    if tag.startswith("N"):
        return wn.NOUN
    elif tag.startswith("V"):
        return wn.VERB
    elif tag.startswith("J"):
        return wn.ADJ
    elif tag.startswith("R"):
        return wn.ADV
    else:
        return None

def classify_swn_gram(text):
    word_tokenized = tokenize_text(text)
    tags = [nltk.pos_tag(s) for s in word_tokenized]
    total_score = 0
    for s in tags:
        for w, t in s:
            synsets = list(swn.senti_synsets(w, tag2wn(t)))
            word_score = 0
            for synset in synsets:
                word_score += (synset.pos_score() - synset.neg_score())
            if len(synsets) > 0:
                total_score += word_score / len(synsets)
    return total_score

In [1050]:
# Opinión 1
print(classify_swn_gram(opinion1))

0.5139880952380951


In [1051]:
# Opinión 2
print(classify_swn_gram(opinion2))

-0.05469148771028469


In [1052]:
# Opinión 3
print(classify_swn_gram(opinion3))

0.7655677655677655


Notamos que la mejora es más bien leve.

---
- **Ejercicio 3:** Se pide extender lo que se ha hecho en los ejercicios anteriores, pero ampliado para todas las opiniones contenidas en un fichero .csv (textsSentimentsPNN.csv), donde cada opinión está anotada con su polaridad (positiva, negativa o neutra). Un ejemplo del formato del archivo se muestra a continuación, en la primera columna estaría el texto de la opinión y en la segunda el sentimiento (Positive, Negative o Neutral).  
 
    Utilizando de nuevo SentiWordNet se pide predecir la polaridad de cada mensaje. Una vez que se tenga la polaridad de cada mensaje, utilizar las métricas del paquete sklearn.metrics de la librería scikit-learn para obtener los valores de accuracy, precisión, recall y f-measure. En definitiva, se quiere poder cuantificar si se están clasificando bien las opiniones según su polaridad. Cuanto más cercano a 1 sea el valor que se obtiene con estas métricas, mejor estará clasificando cada mensaje.

In [1053]:
text_sentiments = pd.read_csv("data/textsSentimentsPNN.csv")
text_sentiments.head()

,Text,Sentiment
0,Enjoying a beautiful day at the park!,Positive
1,Traffic was terrible this morning.,Negative
2,Just finished an amazing workout! 💪,Positive
3,Excited about the upcoming weekend getaway!,Positive
4,Trying out a new recipe for dinner tonight.,Neutral


In [1054]:
# Sustituimos:
# Positive -> 1
# Neutral -> 0
# Negative -> 2
positive = text_sentiments["Sentiment"] == "Positive"
neutral = text_sentiments["Sentiment"] == "Neutral"
negative = text_sentiments["Sentiment"] == "Negative"

text_sentiments.loc[positive, "Sentiment"] = 1
text_sentiments.loc[neutral, "Sentiment"] = 0
text_sentiments.loc[negative, "Sentiment"] = 2

text_sentiments.head()

,Text,Sentiment
0,Enjoying a beautiful day at the park!,1
1,Traffic was terrible this morning.,2
2,Just finished an amazing workout! 💪,1
3,Excited about the upcoming weekend getaway!,1
4,Trying out a new recipe for dinner tonight.,0


In [1055]:
# Función auxiliar para tokenizar el texto
def preprocess_text(text):
    sent_tokenized = PunktSentenceTokenizer().tokenize(text)
    word_tokenized_no_punct = [regexp_tokenize(s, r"[\w\d]+") for s in sent_tokenized]
    # Eliminamos puntuación y stopwords
    no_punct_no_sw = []
    for s in word_tokenized_no_punct:
        no_punct_no_sw.append([w for w in s if w not in english_sw])

    tagged = [nltk.pos_tag(s) for s in no_punct_no_sw]

    return tagged

# Clasificamos cada texto aplicando WSD previamente
def classify_text(text):
    preprocessed_text = preprocess_text(text)
    total_score = 0
    for s in preprocessed_text:
        sent_score = 0
        for w, t in s:
            senti_synsets = swn.senti_synsets(w, tag2wn(t))
            for senti_synset in senti_synsets:
                if senti_synset is not None:
                    sent_score += (senti_synset.pos_score() - senti_synset.neg_score())
        total_score += sent_score
    return total_score

In [ ]:
# Obtenemos y normalizamos las predicciones
sentiments_pred = [classify_text(text) for text in text_sentiments["Text"]]
min_pred = min(sentiments_pred)
max_pred = max(sentiments_pred)

normalized_pred = [2 * ((pred - min_pred) / (max_pred - min_pred)) - 1 for pred in sentiments_pred]
normalized_pred = np.array(normalized_pred).astype(np.float32)

threshold = 0.1
positive_pred = normalized_pred > threshold
negative_pred = normalized_pred < -threshold
neutral_pred = (normalized_pred >= -threshold) & (normalized_pred <= threshold)
y_pred = normalized_pred.copy()
y_pred[positive_pred] = 1
y_pred[neutral_pred] = 0
y_pred[negative_pred] = 2
y_pred = y_pred.astype(np.uint8)

In [1057]:
y_true = np.array(text_sentiments["Sentiment"], dtype=np.uint8)

print(f"Accuracy: {accuracy_score(y_true, y_pred)}")
print(f"Precision: {precision_score(y_true, y_pred, average="weighted")}")
print(f"Recall: {recall_score(y_true, y_pred, average="weighted")}")
print(f"F-score: {f1_score(y_true, y_pred, average="weighted")}")

Accuracy: 0.1044776119402985
Precision: 0.6753731343283582
Recall: 0.1044776119402985
F-score: 0.09097892888498683


/home/wrst/miniconda3/envs/pln1/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [1058]:
print(y_true)

[1 2 1 1 0 1 1 1 2 0 1 2 1 1 0 1 1 1 0 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [1059]:
print(y_pred)

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [1060]:
print(pred)

[ 4.    -2.625  0.25   0.5    0.5   -1.     2.25   1.375  0.125 -0.375
  1.25   0.125  0.875  1.75   2.375  1.875  2.125 -0.125  0.25  -3.375
 15.     0.875  2.375  0.875  0.5    0.375  0.625 12.125  2.625  0.125
 -1.5    3.     1.     0.375  0.     2.75   3.     0.625  0.5    0.
  0.5    0.75   3.5   12.75   0.75   0.625  0.75   3.25   2.625  1.375
  1.625 -0.125 -0.125  1.75   4.     1.25   0.625  2.875  0.375  1.875
  0.    -0.125  2.25   1.375  1.875  1.625  3.   ]
